<a href="https://colab.research.google.com/github/hwankang/chemometrics-tutorials/blob/master/chemometrics_08_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y tensorflow keras
!git clone https://github.com/hwankang/chemometrics-tutorials

In [ ]:
%cd chemometrics-tutorials
!pip install -r requirements.txt

In [ ]:
# Import the required python packages including 
# the custom Chemometric Model objects
import numpy as np
from sklearn import preprocessing
import pandas as pds
import matplotlib.pyplot as plt
import warnings
from sklearn.exceptions import DataConversionWarning

from pyChemometrics.ChemometricsPLSDA import ChemometricsPLSDA
from pyChemometrics.ChemometricsScaler import ChemometricsScaler
from pyChemometrics.ChemometricsOrthogonalPLSDA import ChemometricsOrthogonalPLSDA

# Use to obtain same values as in the text
np.random.seed(350)

In [ ]:
# Set the data conversion warnings to appear only once to avoid repetition during CV
warnings.filterwarnings("ignore", category=DataConversionWarning)

In [ ]:
# Set the plot backend to support interactive plotting
#%matplotlib notebook

In [ ]:
# Load the dataset
X = np.genfromtxt("./data/X_spectra.csv", delimiter=',', dtype=None)
Y = pds.read_csv("./data/worm_yvars.csv",delimiter=',',dtype=None, header=None)
ppm = np.loadtxt("./data/ppm.csv",delimiter=',')

# Use pandas Categorical type to generate the dummy enconding of the Y vector (0 and 1) 
Y1 = pds.Categorical(Y.iloc[:, 0]).codes
Y2 = pds.Categorical(Y.iloc[:, 1]).codes

In [ ]:
# Plot the spectra in the dataset
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure()
plt.plot(ppm, X.T)
plt.title("X matrix of spectra")
plt.xlabel("$\delta$ppm")
plt.gca().invert_xaxis()
plt.ylabel("Intensity")
plt.show()

In [ ]:
# Select the scaling options: 

# Unit-Variance (UV) scaling:
scaling_object_uv = ChemometricsScaler(scale_power=1)

# Pareto scaling:
scaling_object_par = ChemometricsScaler(scale_power=1/2)

# Mean Centring:
scaling_object_mc = ChemometricsScaler(scale_power=0)

In [ ]:
# Create and fit PLS-DA model
pls_da = ChemometricsPLSDA(n_components=2, x_scaler=scaling_object_uv)
pls_da.fit(X, Y1)

In [ ]:
# Plot the scores
pls_da.plot_scores(color=Y1, discrete=True, label_outliers=True, plot_title=None)

In [ ]:
# Plot the weights and loadings.
# w for weights, p for loadings,
# ws for X rotations (rotated version of w) 
pls_da.plot_model_parameters(parameter='p', component=1)

In [ ]:
# Plot the weights and loadings.
# w for weights, p for loadings,
# ws for X rotations (rotated version of w) 
pls_da.plot_model_parameters(parameter='w', component=1)

# 2) Model selection

In [ ]:
pls_da.scree_plot(X, Y1, total_comps=10)

In [ ]:
# Repeated cross_validation
rep_cv = pls_da.repeated_cv(X, Y1, repeats=5, total_comps=10)

##Outlier detection

In [ ]:
pls_da.plot_scores(label_outliers=True)
pls_da.outlier(X)

In [ ]:
pca_outliers = np.array([36, 100, 106, 113, 117])
X = np.delete(X, pca_outliers, axis=0)
Y1 = np.delete(Y1, pca_outliers, axis=0)
Y2 = np.delete(Y2, pca_outliers, axis=0)

In [ ]:
pls_da.scree_plot(X, Y1, total_comps=10)

In [ ]:
# Repeated cross_validation
rep_cv = pls_da.repeated_cv(X, Y1, repeats=5, total_comps=10)

 Refit the model

In [ ]:
# Refit the model with the selected number of components
#pls_da = ChemometricsPLSDA(n_components=10, x_scaler=scaling_object_uv)
pls_da = ChemometricsPLSDA(n_components=8, x_scaler=scaling_object_uv)
pls_da.fit(X, Y1)

In [ ]:
pls_da.plot_scores(color=Y1, discrete=True)

In [ ]:
# Cross-validated ROC curve
pls_da.cross_validation(X, Y1)
pls_da.plot_cv_ROC()

#permutation test

In [ ]:
permt = pls_da.permutation_test(X, Y1, 1000)

In [ ]:
 np.save('permutations_plsda.npy', permt)
permt = np.load('permutations_plsda.npy', allow_pickle=True)

In [ ]:
# plot the results from the permuation test
pls_da.plot_permutation_test(permt, metric='AUC')
plt.xlabel('AUC')
plt.ylabel('Counts')
print("Permutation p-value for the AUC: {0}".format(permt[1]['AUC']))

In [ ]:
# plot the results from the permuation test
pls_da.plot_permutation_test(permt, metric='Q2Y')
plt.xlabel('Q2Y')
plt.ylabel('Counts')
print("Permutation p-value for the Q2Y: {0}".format(permt[1]['Q2Y']))

# 3) Model interpretation and variable importance

In [ ]:
pls_da.plot_model_parameters('w', component=1, sigma=2, cross_val=True, xaxis=ppm)
plt.gca().invert_xaxis()
plt.gca().set_xlabel('ppm')

In [ ]:
pls_da.plot_model_parameters('VIP', sigma=2, cross_val=True, xaxis=ppm)
plt.gca().invert_xaxis()
plt.gca().set_xlabel('ppm')

In [ ]:
pls_da.plot_model_parameters('beta', sigma=2, cross_val=True, xaxis=ppm)
plt.gca().invert_xaxis()
plt.gca().set_xlabel('ppm')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8, 5))
X_scaled = pls_da.x_scaler.transform(X)

cov_x_y = np.dot(Y1.T - Y1.mean(), X_scaled) / (Y1.shape[0]-1)
cov_x_y = cov_x_y/np.linalg.norm(cov_x_y)

ax[0].plot(cov_x_y, 'orange')
ax[1].plot(pls_da.weights_w[:, 0], 'green')
ax[0].set_xlabel('Normalised $Cov(X_{i}, Y)$')
ax[1].set_xlabel('$w$ for PLS component 1')
fig.show()

In [ ]:
# Same model, but coloured by Age instead of Genotype
pls_da.plot_scores(comps=[1, 2], color=Y2, discrete=True)
pls_da.plot_model_parameters('p', component=2)

##Orthogonal PLS

In [ ]:
# Generate an Orthogonal PLS-DA version of the PLS-DA model fitted
orthogonal_pls_da = ChemometricsOrthogonalPLSDA(ncomps=5, xscaler=scaling_object_uv)
orthogonal_pls_da.fit(X, Y1)

In [ ]:
orthogonal_pls_da.plot_scores(color=Y1, orthogonal_component=1, discrete=True)

In [ ]:
orthogonal_pls_da.plot_scores(color=Y2, orthogonal_component=2, discrete=True, label_outliers=False)

In [ ]:
orthogonal_pls_da.plot_model_parameters('p_pred', orthogonal_component = 1, xaxis=ppm)
plt.gca().invert_xaxis()
# 
orthogonal_pls_da.plot_model_parameters('p_ortho', orthogonal_component = 2, xaxis=ppm)
plt.gca().invert_xaxis()

Permutation p-value for variable ranking

In [ ]:
# Plot empirical null distributions for weights
plt.figure()
plt.hist(permt[0]['Weights_w'][:, 3000, 0], 100)
plt.title("Permuted null distribution for weights (w), component 1, {0} $\delta$ppm".format(ppm[3000]))
plt.show()

plt.figure()
plt.hist(permt[0]['Weights_w'][:, 10, 0], 100)
plt.title("Permuted null distribution for weights (w), component 1, {0} $\delta$ppm".format(ppm[10]))
plt.show()

In [ ]:
# Plot empirical null distributions for loadings
# Notice how these are not unimodal and distributed around 0...
plt.figure()
plt.hist(permt[0]['Loadings_p'][:, 3000, 0], 100)
plt.title("Permuted null distribution for loadings (p), component 1, {0} $\delta$ppm".format(ppm[3000]))
plt.show()

plt.figure()
plt.hist(permt[0]['Loadings_p'][:, 10, 0], 100)
plt.title("Permuted null distribution for loadings (p), component 1, {0} $\delta$ppm".format(ppm[10]))
plt.show()

In [ ]:
# Plot empirical null distributions for regression coefficients
plt.figure()
plt.hist(permt[0]["Beta"][:, 3000], 100)
plt.title(r"Permuted null distribution for $\beta$, {0} $\delta$ppm".format(ppm[3000]))
plt.show()

plt.figure()
plt.hist(permt[0]['Beta'][:, 10], 100)
plt.title(r"Permuted null distribution for $\beta$, {0} $\delta$ppm".format(ppm[10]))
plt.show()

In [ ]:
# Always set *nperms* equal to the number of permutations used before
nperms = permt[0]['R2Y'].size
perm_indx = abs(permt[0]['Beta'].squeeze()) >= abs(pls_da.beta_coeffs.squeeze())
counts = np.sum(perm_indx, axis=0)
beta_pvals = (counts + 1) / (nperms + 1)

perm_indx_W = abs(permt[0]['Weights_w'][:, :, 0].squeeze()) >= abs(pls_da.weights_w[:, 0].squeeze())
counts = np.sum(perm_indx_W, axis=0)
w_pvals = (counts + 1) / (nperms + 1)

In [ ]:
plt.figure()
plt.title(r"p-value distribution for the regression coefficients $\beta$ ")
z = plt.hist(beta_pvals, bins=100, alpha=0.8)
plt.axvline(x=0.05, ymin=0, ymax=max(z[0]), color='r', linestyle='--') 
plt.show()

plt.figure()
plt.title(r"p-value distribution for the weights corresponding to the first component")
z = plt.hist(w_pvals, bins=100, alpha=0.8)
plt.axvline(x=0.05, ymin=0, ymax=max(z[0]), color='r', linestyle='--') 
plt.show()

In [ ]:
signif_bpls_idx = np.where(beta_pvals <= 0.05)[0]

print("Number of significant values: {0}".format(len(signif_bpls_idx)))

Comparison between variables highlighted in a multivariate PLS-DA with a univariate analysis

In [ ]:
#load the results of the univariate testing procedure
univ_gen = pds.read_csv('./data/UnivariateAnalysis_Genotype.csv')

# Select significant peaks from univariate analysis 
signif = np.where(univ_gen['genotype_q-value'] < 0.05)[0]

In [ ]:
# p-values significant for association with genotype in both the PLS analysis and linear regression
common_idx = np.array([x for x in signif_bpls_idx if x in signif])
# p-values significant only in PLS
pls_idx = np.array([x for x in signif_bpls_idx if x not in signif])
# p-values significant only for linear regression
reg_idx = np.array([x for x in signif if x not in signif_bpls_idx])

In [ ]:
plt.figure()
plt.plot(ppm, X.mean(axis=0))
#plt.scatter(ppm[signif], X.mean(axis=0)[signif], c='red', s=30)
plt.scatter(ppm[reg_idx], X.mean(axis=0)[reg_idx], c='red', s=30)
plt.scatter(ppm[pls_idx], X.mean(axis=0)[pls_idx], c='orange', s=30)
plt.scatter(ppm[common_idx], X.mean(axis=0)[common_idx], c='green', s=30)
plt.gca().invert_xaxis()
plt.legend(['Mean Spectrum', 'Both', 'Linear regression only', 'PLS only'])
plt.show()